In [1]:
import os
print(1)  # Line 1
import re
print(2)  # Line 2
import cv2
print(3)  # Line 3
import tarfile
print(4)  # Line 4
import datetime
print(5)  # Line 5
import warnings
print(6)  # Line 6
import prettytable
print(7)  # Line 7
import numpy as np
print(8)  # Line 8
import pandas as pd
print(9)  # Line 9
from PIL import Image
print(10) # Line 10
import seaborn as sns
print(11) # Line 11
from tqdm import tqdm
print(12) # Line 12
import tensorflow as tf
print(13) # Line 13
from tensorflow import keras
print(14) # Line 14
from tensorflow import concat
print(15) # Line 15
from tensorflow import repeat
print(16) # Line 16
from collections import Counter
print(17) # Line 17
import matplotlib.pyplot as plt
print(18) # Line 18
import matplotlib.image as mpimg
print(19) # Line 19
warnings.filterwarnings('ignore')
print(20) # Line 20
from sklearn.utils import shuffle
print(21) # Line 21
from prettytable import PrettyTable
print(22) # Line 22
from skimage.transform import resize
print(23) # Line 23
import nltk.translate.bleu_score as bleu
print(24) # Line 24
from tensorflow.keras.models import Model
print(25) # Line 25
from tensorflow.keras.models import load_model
print(26) # Line 26
from tensorflow.keras.backend import expand_dims 
print(27) # Line 27
from nltk.translate.bleu_score import sentence_bleu
print(28) # Line 28
from tensorflow.keras.layers import TimeDistributed
print(29) # Line 29
from tensorflow.keras.applications import DenseNet121
print(30) # Line 30
from tensorflow.keras.preprocessing.text import Tokenizer
print(31) # Line 31
from tensorflow.keras.layers import concatenate, Concatenate
print(32) # Line 32
from tensorflow.keras.preprocessing.image import img_to_array
print(33) # Line 33
from tensorflow.keras.preprocessing.sequence import pad_sequences
print(34) # Line 34
from tensorflow.keras.applications.densenet import preprocess_input
print(35) # Line 35
from tensorflow.keras.layers import Input, Softmax, RNN, Dense, Embedding, LSTM, Layer, Dropout, GRU
print(36) # Line 36


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36


In [2]:
image_model = DenseNet121(weights='CheXNet_weights.h5', classes = 14, input_shape=(256,256,3))
model = Model(image_model.input, image_model.layers[-2].output)
print("Done")

Done


In [13]:
dataframe = pd.read_csv('dataframe.csv')
dataframe.head()

,image_name,image_caption,comparison,indication,findings,impression,image_count,indication_count,findings_count,impression_count
0,"CXR1_1_IM-0001-3001.png,CXR1_1_IM-0001-4001.png",ray chest pa and lateral,none,positive tb test,the cardiac silhouette and mediastinum size ar...,normal chest x,2,3,33,3
1,"CXR10_IM-0002-2001.png,CXR10_IM-0002-1001.png",pa and lateral chest x,chest radiographs,male chest pain,the cardiomediastinal silhouette is within nor...,no acute cardiopulmonary process,2,3,38,4
2,"CXR100_IM-0002-1001.png,CXR100_IM-0002-2001.png",chest frontal lateral pm,none,no indication,both lungs are clear and expanded heart and me...,no active disease,2,2,10,3
3,"CXR1000_IM-0003-1001.png,CXR1000_IM-0003-2001....",pa and lateral chest x,pa and lateral chest radiographs,male,there is increased opacity within the right up...,increased opacity in the right upper lobe with...,3,1,52,36
4,"CXR1001_IM-0004-1002.png,CXR1001_IM-0004-1001.png",chest frontal lateral pm,none,dyspnea subjective fevers arthritis immigrant ...,interstitial markings are diffusely prominent ...,diffuse fibrosis no visible focal acute disease,2,7,14,7


In [5]:
projections = pd.read_csv("indiana_projections.csv")
projections.head()

,uid,filename,projection
0,1,1_IM-0001-4001.dcm.png,Frontal
1,1,1_IM-0001-3001.dcm.png,Lateral
2,2,2_IM-0652-1001.dcm.png,Frontal
3,2,2_IM-0652-2001.dcm.png,Lateral
4,3,3_IM-1384-1001.dcm.png,Frontal


In [6]:
def front_lateral(lst):
    ''' This Function check the image projection of the different images
    and return frontal or lateral of the given image '''
    frontal = []
    lateral = ""
    for img in lst:
        projection = projections[projections['filename'].str.contains(re.search(r"\d.*\_IM-\d.*\.", img).group())]['projection'].values
        if projection == "Lateral":
            lateral = img
        else:
            frontal.append(img)
    return frontal, lateral

In [18]:
columns = ["front X-Ray", "lateral X-Ray", "findings"]

df = pd.DataFrame(columns = columns)
#Creating diffrent dataframe for single images, 
#to distribute it in all train test validate dataset in equal proportion.
SingleImages_df = pd.DataFrame(columns = columns)

for item in tqdm(dataframe.iterrows()):
    img_lst = item[1]['image_name'].split(',')

    if len(img_lst) > 2:
        #check projection of x-ray images
        frontal, lateral = front_lateral(img_lst)
        
        #If no lateral x-ray is present
        if lateral == "":
            frontal, lateral = frontal[:-1], frontal[-1]
        
        #If more than one frontal image is present 
        for i in frontal:
            xray_front = i
            xray_lat = lateral
            df = df._append(pd.Series([xray_front, xray_lat, item[1]['findings']], index = columns), ignore_index = True)

    elif len(img_lst) == 2:
        xray_front = img_lst[0] 
        xray_lat = img_lst[1]
        df = df._append(pd.Series([xray_front, xray_lat, item[1]['findings']], index = columns), ignore_index = True)

    elif len(img_lst) == 1:
        xray_front = img_lst[0] 
        xray_lat = img_lst[0]
        SingleImages_df = SingleImages_df._append(pd.Series([xray_front, xray_lat, item[1]['findings']], index = columns), ignore_index = True)

3851it [00:04, 819.64it/s]


In [20]:
from sklearn.model_selection import train_test_split

train1, test1  = train_test_split(df, test_size = 0.10, random_state = 42, shuffle = True)
train1, valid1 = train_test_split(train1, test_size = 0.10, random_state = 42, shuffle = True)

print('train data :', train1.shape)
print('test data  :', test1.shape)
print('valid data :', valid1.shape)

train data : (2860, 3)
test data  : (354, 3)
valid data : (318, 3)


In [21]:
train2, test2  = train_test_split(SingleImages_df, test_size = 0.10, random_state = 42, shuffle = True)
train2, valid2 = train_test_split(train2, test_size = 0.10, random_state = 42, shuffle = True)


print('train data :', train2.shape)
print('test data  :', test2.shape)
print('valid data :', valid2.shape)

train data : (360, 3)
test data  : (45, 3)
valid data : (41, 3)


In [22]:
train = np.append(train1, train2, axis=0)
test = np.append(test1, test2, axis=0)
valid = np.append(valid1, valid2, axis=0)
print("===== Final data point shape =====")
train.shape, test.shape, valid.shape

===== Final data point shape =====


((3220, 3), (399, 3), (359, 3))

In [23]:
# Set a random state
from sklearn.utils import shuffle
for i in range(3):
    train = shuffle(train, random_state=15)
    test = shuffle(test, random_state=15)
    valid = shuffle(valid, random_state=15)

In [24]:
train= train[:-20,:]
train.shape

(3200, 3)

In [25]:
validation = valid[:-9,:]
validation.shape

(350, 3)

In [26]:
columns = ["front X-Ray", "lateral X-Ray", "findings"]

train = pd.DataFrame(train, columns = columns)
test = pd.DataFrame(test, columns = columns)
validation = pd.DataFrame(validation, columns = columns)

In [27]:
def decoder_io(df):
  '''This function Add start and end token in finding feature and prepare decoder input with output
  It takes dataframe(df) as parameter and returns df after applying necessary operations. '''
  
  df['dec_ip'] = '<start>' + ' ' + df.findings.astype(str)  #Decoder input
  df['dec_op'] = df.findings.astype(str) + ' ' +'<end>'     #Decoder output
  df['findings']      = '<start> ' + df.findings + ' <end>'
  return df

In [28]:
train = decoder_io(train)
validation = decoder_io(validation)
test = decoder_io(test)

In [29]:
train.head()

,front X-Ray,lateral X-Ray,findings,dec_ip,dec_op
0,CXR677_IM-2249-2001.png,CXR677_IM-2249-1001.png,<start> picc line catheter tip in the right at...,<start> picc line catheter tip in the right at...,picc line catheter tip in the right atrium hea...
1,CXR1096_IM-0066-2001.png,CXR1096_IM-0066-2001.png,<start> the heart size is within normal limits...,<start> the heart size is within normal limits...,the heart size is within normal limits no foca...
2,CXR143_IM-0276-2001.png,CXR143_IM-0276-1001.png,<start> the lungs are clear no pleural effusio...,<start> the lungs are clear no pleural effusio...,the lungs are clear no pleural effusion is see...
3,CXR3848_IM-1946-1001-0001.png,CXR3848_IM-1946-1001-0002.png,<start> the heart size is moderate to severely...,<start> the heart size is moderate to severely...,the heart size is moderate to severely enlarge...
4,CXR317_IM-1493-1001.png,CXR317_IM-1493-2001.png,<start> normal heart size normal mediastinal s...,<start> normal heart size normal mediastinal s...,normal heart size normal mediastinal silhouett...


In [30]:
image_model = DenseNet121(weights='CheXNet_weights.h5', classes = 14, input_shape=(256,256,3))

In [31]:
model = Model(image_model.input, image_model.layers[-2].output)

In [34]:
def load_image(img_name):
  """Loads image in array format"""

  image = Image.open(img_name)
  X = np.asarray(image.convert("RGB"))
  X = np.asarray(X)
  X = preprocess_input(X)
  X = resize(X, (256,256,3))
  X = np.expand_dims(X, axis=0)
  X = np.asarray(X)
    
  return X

In [43]:
def image(df):
  '''
    input -- dataframe(df)
    output -- dataframe(df)
    process - convert images into 256 X 256, then using CHeXNET model generate tensor(concate two image tensor)

  '''
  path = 'images/'
  image1_paths = df['front X-Ray'].astype(str).tolist()
  image2_paths = df['lateral X-Ray'].astype(str).tolist()

  image_features = []
  for i in tqdm(range(len(image1_paths))):

    i1 = load_image(path+image1_paths[i])
    i2 = load_image(path+image2_paths[i])
    img1_features = model.predict(i1)
    img2_features = model.predict(i2)
    img1_features = np.vstack(img1_features).astype(float)
    img2_features = np.vstack(img2_features).astype(float)
    
    tensor = np.concatenate((img1_features, img2_features), axis=1)

    image_features.append(tensor)
  df['image_features'] =image_features
  return df

In [40]:
cwd = os.getcwd()

In [44]:
train = image(train)
test = image(test)
validation = image(validation)

  0%|                                                                                         | 0/3200 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step


  0%|                                                                                 | 1/3200 [00:00<28:22,  1.88it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step


  0%|                                                                                 | 2/3200 [00:01<27:35,  1.93it/s]

...

 99%|███████████████████████████████████████████████████████████████████████████████▌| 348/350 [02:59<00:01,  1.99it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step


100%|███████████████████████████████████████████████████████████████████████████████▊| 349/350 [03:00<00:00,  1.99it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step


100%|████████████████████████████████████████████████████████████████████████████████| 350/350 [03:00<00:00,  1.94it/s]


In [45]:
train_np = train.to_numpy()
test_np = test.to_numpy()
validation_np = validation.to_numpy()

np.save('train', train_np)
np.save('test', test_np)
np.save('validation', validation_np)

In [46]:
train.head()

,front X-Ray,lateral X-Ray,findings,dec_ip,dec_op,image_features
0,CXR677_IM-2249-2001.png,CXR677_IM-2249-1001.png,<start> picc line catheter tip in the right at...,<start> picc line catheter tip in the right at...,picc line catheter tip in the right atrium hea...,"[[1.3660428521689028e-05, 0.001198855228722095..."
1,CXR1096_IM-0066-2001.png,CXR1096_IM-0066-2001.png,<start> the heart size is within normal limits...,<start> the heart size is within normal limits...,the heart size is within normal limits no foca...,"[[0.00015775891370140016, 0.001071828184649348..."
2,CXR143_IM-0276-2001.png,CXR143_IM-0276-1001.png,<start> the lungs are clear no pleural effusio...,<start> the lungs are clear no pleural effusio...,the lungs are clear no pleural effusion is see...,"[[0.00012958170555066317, 0.001051775994710624..."
3,CXR3848_IM-1946-1001-0001.png,CXR3848_IM-1946-1001-0002.png,<start> the heart size is moderate to severely...,<start> the heart size is moderate to severely...,the heart size is moderate to severely enlarge...,"[[0.0003334000357426703, 0.0013413153355941176..."
4,CXR317_IM-1493-1001.png,CXR317_IM-1493-2001.png,<start> normal heart size normal mediastinal s...,<start> normal heart size normal mediastinal s...,normal heart size normal mediastinal silhouett...,"[[0.00026678695576265454, 0.001648269011639058..."
